## Where are the rail routes?

In [1]:
import _section1_utils as section1
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import portfolio_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
import sys

sys.path.append("../gtfs_funnel/")
import route_typologies

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
org_name_lists = ["Capitol Corridor Joint Powers Authority"]

In [5]:
organization_name = "Capitol Corridor Joint Powers Authority"

In [6]:
analysis_date_list = ["2024-11-13"]

In [7]:
analysis_date = "2024-11-13"

In [8]:
schd_keys = [
    "f5a749dd65924e025b1293c58f95f8d6",
]

In [9]:
name_list = ["Bay Area 511 Capitol Corridor Schedule"]

In [10]:
ROUTE_TYPOLOGIES = GTFS_DATA_DICT.schedule_tables.route_typologies
route_typologies_df = pd.read_parquet(
    f"{SCHED_GCS}{ROUTE_TYPOLOGIES}_{analysis_date}.parquet"
)

In [11]:
route_typologies_df.head(1)

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,3,3407,Route 18,Route 18,Route 18__Route 18,0,1,0,0,0.00,18827,OHS TRIPPER - C ST - DORIS ST,8611.13,1,0


### Fix `operator_profile`
* Nothing shows up for `# Rail Route Types` even though there's one rail route.

In [12]:
operator_profiles = section1.load_operator_ntd_profile(organization_name)

In [13]:
operator_profiles

,schedule_gtfs_dataset_key,VP per Minute (All Routes),Spatial Accuracy (All Routes),Date,# Routes,# Trips,# Shapes,# Stops,# Arrivals,Operator Service Miles,Avg Arrivals per Stop,# Downtown Local Route Types,# Local Route Types,# Coverage Route Types,# Rapid Route Types,# Express Route Types,# Rail Route Types,Transit Operator,Organization ID,Organization,District,counties_served,service_area_sq_miles,hq_city,service_area_pop,organization_type,primary_uza_name,reporter_type
13,f5a749dd65924e025b1293c58f95f8d6,2.57,0.00,2024-12-11,3.00,57.00,13.00,20.00,378.00,233.57,18.90,0.00,0.00,1.00,0.00,0.00,0.00,Bay Area 511 Capitol Corridor Schedule,recvEBkSBc7UxlarC,Capitol Corridor Joint Powers Authority,04 - Oakland,None,<NA>,None,<NA>,None,None,None


In [14]:
f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/digest/operator_profiles.parquet'

#### Go to `gtfs_digest/merge_operator_data/concatenate_rt_vs_schedule_operator_metrics`

In [15]:
op_rt_sched_metrics = merge_operator_data.concatenate_rt_vs_schedule_operator_metrics(
    analysis_date_list
)

In [16]:
op_rt_sched_metrics.head(1)

,schedule_gtfs_dataset_key,operator_n_routes,operator_n_trips,operator_n_shapes,operator_n_stops,operator_n_arrivals,operator_route_length_miles,operator_arrivals_per_stop,n_downtown_local_routes,n_local_routes,n_coverage_routes,n_rapid_routes,n_express_routes,n_rail_routes,name,organization_source_record_id,organization_name,service_date
0,0139b1253130b33adcd4b3a4490530d2,25,310,132,308,4859,664.70,15.78,0,0,24,14,0,0,TCRTA TripShot Schedule,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,2024-11-13


In [17]:
op_rt_sched_metrics = op_rt_sched_metrics.loc[
    op_rt_sched_metrics.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [18]:
op_rt_sched_metrics

,schedule_gtfs_dataset_key,operator_n_routes,operator_n_trips,operator_n_shapes,operator_n_stops,operator_n_arrivals,operator_route_length_miles,operator_arrivals_per_stop,n_downtown_local_routes,n_local_routes,n_coverage_routes,n_rapid_routes,n_express_routes,n_rail_routes,name,organization_source_record_id,organization_name,service_date
178,f5a749dd65924e025b1293c58f95f8d6,3,57,13,20,378,233.57,18.90,0,0,1,0,0,0,Bay Area 511 Capitol Corridor Schedule,recvEBkSBc7UxlarC,Capitol Corridor Joint Powers Authority,2024-11-13


#### File I need to correct is in `data-analyses/gtfs_funnel/operator_scheduled_stats.py`

In [19]:
GTFS_DATA_DICT.schedule_tables.operator_scheduled_stats

'operator_profiles/operator_scheduled_stats'

In [20]:
ROUTE_TYPOLOGY = GTFS_DATA_DICT.schedule_tables.route_typologies

#### I thought I ran this yesterday

In [21]:
route_typology = pd.read_parquet(f"{SCHED_GCS}{ROUTE_TYPOLOGY}_{analysis_date}.parquet")

In [22]:
analysis_date

'2024-11-13'

In [23]:
route_typology = route_typology.loc[route_typology.name.isin(name_list)]

In [24]:
route_typology

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_coverage,is_downtown_local
1505,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn,0,0,0,0,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,1,0


### Only one route appears.

### Check out `gtfs_funnel/route_typologies/categorize_routes_by_name`
* 3 routes appear. One is rail.
* All 3 routes are retained until the end.

In [25]:
df = helpers.import_scheduled_trips(
    analysis_date,
    columns=[
        "gtfs_dataset_key",
        "name",
        "route_type",
        "route_id",
        "route_long_name",
        "route_short_name",
    ],
    get_pandas=True,
)

In [26]:
df = df.loc[df.name.isin(name_list)]

In [27]:
df = df.assign(
    route_id=df.route_id.fillna(""),
    route_short_name=df.route_short_name.fillna(""),
    route_long_name=df.route_long_name.fillna(""),
)

In [28]:
df = df.assign(combined_name=df.route_short_name + "__" + df.route_long_name)

In [29]:
df

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name
1122,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn
1123,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,SF,Shuttle_SF,SF,SF__Shuttle_SF
1124,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor


In [30]:
def tag_rapid_express_rail(route_name_string: str, route_type_string: str) -> pd.Series:
    """
    Use the combined route_name and see if we can
    tag out words that indicate the route is
    express, rapid, local, and rail.

    Treat rail as own category.
    For local routes, we'll pass that through NACTO to see
    if we can better categorize as downtown_local, local, or coverage.
    """
    route_name_string = route_name_string.lower()

    express = 0
    rapid = 0
    rail = 0

    if any(substring in route_name_string for substring in ["express", "limited"]):
        express = 1
    if "rapid" in route_name_string:
        rapid = 1

    rail_types = ["0", "1", "2", "5", "6", "7", "11", "12"]
    if route_type_string in rail_types:
        rail = 1

    return pd.Series(
        [express, rapid, rail], index=["is_express", "is_rapid", "is_rail"]
    )

In [31]:
typology_tags = df.apply(
    lambda x: tag_rapid_express_rail(x.combined_name, x.route_type), axis=1
)

In [32]:
df2 = pd.concat([df, typology_tags], axis=1)

In [33]:
df2 = df2.assign(
    is_local=df2.apply(
        lambda x: 1
        if (x.is_express == 0) and (x.is_rapid == 0) and (x.is_rail == 0)
        else 0,
        axis=1,
    ).astype(int)
)

In [34]:
df2

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local
1122,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn,0,0,0,1
1123,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,SF,Shuttle_SF,SF,SF__Shuttle_SF,0,0,0,1
1124,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,0,1,0


### `prep_roads`

In [35]:
roads = route_typologies.prep_roads(GTFS_DATA_DICT)

### `overlay_shapes_to_roads`
* The other 2 routes disappear here. 
* Break it out here. 

In [36]:
ROAD_BUFFER_METERS = 20

#### `common_shape` doesn't have the rail routes -> go to `segment_speed_utils/gtfs_schedule_wrangling` to find it.

In [37]:
route_dir_cols = ["gtfs_dataset_key", "route_id", "direction_id"]

keep_trip_cols = route_dir_cols + ["trip_instance_key", "shape_id", "shape_array_key"]

trips = helpers.import_scheduled_trips(
    analysis_date, columns=keep_trip_cols, get_pandas=True
).rename(columns={"schedule_gtfs_dataset_key": "gtfs_dataset_key"})
sorting_order = [True for i in route_dir_cols]

In [38]:
trips = trips.loc[trips.gtfs_dataset_key.isin(schd_keys)]

#### `CC` is the rail.

In [39]:
trips[["route_id"]].drop_duplicates()

,route_id
17474,Shuttle
17475,SF
17491,CC


In [40]:
most_common_shape = (
    trips.groupby(
        route_dir_cols + ["shape_id", "shape_array_key"],
        observed=True,
        group_keys=False,
        dropna=False,
    )
    .agg({"trip_instance_key": "count"})
    .reset_index()
    .sort_values(
        route_dir_cols + ["trip_instance_key"], ascending=sorting_order + [False]
    )
    .drop_duplicates(subset=route_dir_cols)
    .reset_index(drop=True)[route_dir_cols + ["shape_id", "shape_array_key"]]
).rename(
    columns={
        "gtfs_dataset_key": "schedule_gtfs_dataset_key",
        "shape_id": "common_shape_id",
    }
)

In [41]:
most_common_shape[["route_id"]].drop_duplicates()

,route_id
0,CC
2,SF
4,Shuttle


In [42]:
shape_geom = helpers.import_scheduled_shapes(
    analysis_date,
    columns=["shape_array_key", "geometry"],
)

In [43]:
common_shape_geom = pd.merge(
    shape_geom, most_common_shape, on="shape_array_key", how="inner"
).drop(columns="shape_array_key")

In [44]:
common_shape_geom.drop(columns=["geometry"])

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id
0,f5a749dd65924e025b1293c58f95f8d6,CC,1.00,92232e9e-55e8-4f8a-bb86-1aeb515be83e
1,f5a749dd65924e025b1293c58f95f8d6,CC,0.00,5cf6811a-2f53-4199-b315-4408eb816e82
2,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3
3,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00,e70e22aa-8bb7-44eb-b12c-f98aeb8f61b7
4,f5a749dd65924e025b1293c58f95f8d6,SF,1.00,ejnn
5,f5a749dd65924e025b1293c58f95f8d6,SF,0.00,cpz2


In [45]:
route_info = (
    helpers.import_scheduled_trips(
        analysis_date,
        columns=[
            "gtfs_dataset_key",
            "route_id",
            "route_long_name",
            "route_short_name",
            "route_desc",
        ],
    )
    .drop_duplicates()
    .pipe(portfolio_utils.add_route_name)
    .drop(columns=["route_long_name", "route_short_name", "route_desc"])
)

In [46]:
route_info.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'route_name_used'], dtype='object')

In [47]:
route_info = route_info.loc[route_info.schedule_gtfs_dataset_key.isin(schd_keys)]

In [48]:
route_info

,schedule_gtfs_dataset_key,route_id,route_name_used
1122,f5a749dd65924e025b1293c58f95f8d6,Shuttle,Shuttle to Auburn
1123,f5a749dd65924e025b1293c58f95f8d6,SF,Shuttle to San Francisco Transbay Terminal
1124,f5a749dd65924e025b1293c58f95f8d6,CC,"Daily train service between Auburn, Sacramento, Oakland and San Jose"


In [49]:
common_shape_geom2 = pd.merge(
    common_shape_geom,
    route_info.rename(columns={"route_name_used": "route_name"}),
    on=["schedule_gtfs_dataset_key", "route_id"],
)

In [50]:
common_shape_geom2[["route_id"]].drop_duplicates()

,route_id
0,CC
2,Shuttle
4,SF


In [51]:
common_shape = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
    analysis_date
)

In [52]:
common_shape.columns

Index(['geometry', 'schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'common_shape_id', 'route_name'],
      dtype='object')

In [53]:
common_shape = common_shape.loc[common_shape.schedule_gtfs_dataset_key.isin(schd_keys)]

In [54]:
common_shape[["route_id", "direction_id"]].drop_duplicates()

,route_id,direction_id
318,CC,1.00
319,CC,0.00
756,Shuttle,1.00
757,Shuttle,0.00
902,SF,1.00
903,SF,0.00


In [55]:
common_shape.explore("route_id")

In [56]:
common_shape_piped = gtfs_schedule_wrangling.most_common_shape_by_route_direction(
    analysis_date
).pipe(helpers.remove_shapes_outside_ca)

In [57]:
common_shape_piped = common_shape_piped.loc[
    common_shape_piped.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [58]:
common_shape_piped[["route_id"]].drop_duplicates()

,route_id
756,Shuttle


#### Continuing with `gtfs_funnel/route_typologies` the `if==main`

In [59]:
ROAD_BUFFER_METERS = 20

In [62]:
TYPOLOGY_THRESHOLD = 0.10

In [60]:
gdf = route_typologies.overlay_shapes_to_roads(roads, analysis_date, ROAD_BUFFER_METERS)

In [69]:
gdf.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'common_shape_id', 'route_name', 'route_meters', 'freq_category',
       'typology', 'pct_typology'],
      dtype='object')

In [70]:
gdf = gdf.loc[gdf.schedule_gtfs_dataset_key.isin(schd_keys)]

In [71]:
gdf.route_id.nunique()

3

In [94]:
gdf[["route_id", "direction_id", "typology", "pct_typology"]].drop_duplicates().sort_values(
    by=["route_id","direction_id", "pct_typology"]
)

,route_id,direction_id,typology,pct_typology
25633,CC,0.00,downtown_local,0.00
25634,CC,0.00,local,0.00
25637,CC,0.00,rapid,0.00
25636,CC,0.00,coverage,0.01
25635,CC,0.00,rapid,0.02
25639,CC,1.00,downtown_local,0.00
25640,CC,1.00,local,0.00
25641,CC,1.00,rapid,0.00
25642,CC,1.00,coverage,0.01
25647,SF,0.00,rapid,0.01


#### Filtering out for typology_threshold throws away the other routes.

In [80]:
route_typology_df = gdf.loc[gdf.pct_typology >= TYPOLOGY_THRESHOLD]

In [85]:
route_typology_df

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id,route_name,route_meters,freq_category,typology,pct_typology
25664,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,low,coverage,0.16


In [86]:
route_typology_df2 = route_typologies.primary_secondary_typology(route_typology_df)

In [87]:
route_typology_df2.shape

(1, 7)

In [88]:
route_typology_df2.head(1)

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id,route_name,route_meters,is_nacto_coverage
0,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,1


In [89]:
route_typology_df2

,schedule_gtfs_dataset_key,route_id,direction_id,common_shape_id,route_name,route_meters,is_nacto_coverage
0,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,1


In [90]:
route_tagged = route_typologies.categorize_routes_by_name(analysis_date)

#### All the routes pop back here.

In [92]:
route_tagged = route_tagged.loc[route_tagged.schedule_gtfs_dataset_key.isin(schd_keys)]

In [93]:
route_tagged

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local
1122,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn,0,0,0,1
1123,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,SF,Shuttle_SF,SF,SF__Shuttle_SF,0,0,0,1
1124,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,0,1,0


In [97]:
df3 = pd.merge(
            route_tagged,
            route_typology_df2,
            on = ["schedule_gtfs_dataset_key", "route_id"],
            how = "left"
        )

In [101]:
df3

,schedule_gtfs_dataset_key,name,route_type,route_id,route_long_name,route_short_name,combined_name,is_express,is_rapid,is_rail,is_local,direction_id,common_shape_id,route_name,route_meters,is_nacto_coverage
0,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,Shuttle,Shuttle_Auburn,Shuttle,Shuttle__Shuttle_Auburn,0,0,0,1,1.00,104dc91b-4f7b-4f93-bea2-ebfe9adabcf3,Shuttle to Auburn,60220.51,1.00
1,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,3,SF,Shuttle_SF,SF,SF__Shuttle_SF,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2,f5a749dd65924e025b1293c58f95f8d6,Bay Area 511 Capitol Corridor Schedule,2,CC,Capitol Corridor,CC,CC__Capitol Corridor,0,0,1,0,NaN,NaN,NaN,NaN,NaN


In [100]:
df3.pipe(route_typologies.reconcile_route_and_nacto_typologies)

KeyError: "['is_nacto_rapid'] not in index"